# ASMSA: Prepare and check input files

**Next steps**
- [tune.ipynb](tune.ipynb): Perform initial hyperparameter tuning for this molecule
- [train.ipynb](train.ipynb): Use results of previous tuning in more thorough training
- [md.ipynb](md.ipynb): Use a trained model in MD simulation with Gromacs

In [ ]:
#avoid TF to consume GPU memory
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
tf.config.list_logical_devices()

import torch

In [ ]:
import mdtraj as md
import numpy as np
import urllib.request
import asmsa
import os
import re
import tensorflow as tf
import gromacs as gmx
import gromacs.fileformats as gf
import matplotlib.pyplot as plt
import numpy as np

## Prepare input files

In [ ]:
# Define input files
base = '../DE-Shaw/trpcage_correct'

# input conformation, it should not contain hydrogens
conf = base + '.pdb'

# input trajectory
# atom numbering must be consistent with {conf}, no hydrogens as well

traj = '../DE-Shaw/trpcage_red.xtc'

# everything else is generated with pdb2gmx to make sure the files are consistent

#### Generate important files

   * **Density of additional internal coordinates**: how many randomly sampled distances from all atom-to-atom one atom should appear in average
   * **Inputs file generation**: file to be used in the next notebooks
   * **pdb2gmx**: convert a PDB file into a GROMACS topology and generate a coordinate file, topology file and index file

In [ ]:
nb_density = 2 # integer in [1, n_atoms-1]

In [ ]:
topol = base + '.top'
index = base + '.ndx'
gro = base + '.gro'

with open('inputs.py','w') as i:
    i.write(f'''
base = '{base}'
conf = '{conf}'
traj = '{traj}'
topol = '{topol}'
index = '{index}'
gro = '{gro}'

nb_density = {nb_density}
''')

In [ ]:
gmx.pdb2gmx(f=conf, ignh=True,p=topol,n=index,o=gro,water='tip3p',ff='amber99sb-ildn')

### Sanity checks:
 * **Check**: if everything in your trajectory is fine, before proceeding

In [ ]:
# Load the trajectory, it should report expected numbers of frames and atoms/residua

tr = md.load(traj,top=conf)
idx=tr[0].top.select("name CA")

# for trivial cases like Ala-Ala, where superposing on CAs fails
#idx=tr[0].top.select("element != H")

tr.superpose(tr[0],atom_indices=idx)

In [ ]:
# Visual check, all frames should look "reasonable"

# Because of different conventions of numbering atoms in proteins,
# PDB file {conf} and the trajectory {traj} can become inconsistent, and this would appear here 
# as rather weird shapes of the molecule

import nglview as nv

v = nv.show_mdtraj(tr)
v.clear()
v.add_representation("licorice")
v.add_representation("cartoon")
v

## Datasets preprocessing

In [ ]:
tr_orig = tr

### Shuffle configurations in trajectory

In [ ]:
# shuffle the trajectory so the configurations are dispersed across all datasets
np.random.shuffle(tr.xyz)

### Select proportions to divide the trajectory

In [ ]:
# - set proportions for train, validation and test datasets
# - proportions must be equal to 1 when added together
train = .7
validation = .15
test = .15

assert train + validation + test == .9999999999999999 or 1

tr_i = len(tr) * train
X_train = tr.slice(slice(0,int(tr_i)))

va_i = len(tr) * validation
X_validate = tr.slice(slice(int(tr_i),int(tr_i)+int(va_i)))

te_i = len(tr) * test
X_test = tr.slice(slice(int(tr_i)+int(va_i),len(tr)))

X_train.xyz.shape, X_validate.xyz.shape, X_test.xyz.shape

### Divide the trajectory


In [ ]:
X_train.save_xtc('train.xtc')
X_validate.save_xtc('validate.xtc')
X_test.save_xtc('test.xtc')

#### Recovery

In [ ]:
# eventual recovery

X_train = md.load_xtc('train.xtc',conf)
X_validate = md.load_xtc('validate.xtc',conf)
X_test = md.load_xtc('test.xtc',conf)


### Compute RMSD between
   * **train x validation** trajectory and filter similar structures in train trajectory
   * **train x test** trajectory and filter similar structures in train trajectory
   * **test x validation** trajectory and filter similar structures in test trajectory

In [ ]:
# get RMSD from train trajectory compared to validation trajectory
gmx.select(s=conf,on='backbone.ndx',select='Backbone')

In [ ]:
gmx.rms(s=conf,f='train.xtc',f2='validate.xtc',n='backbone.ndx',m='trainxval_rmsd.xpm')

In [ ]:
# load the RMDS matrix
txv = gf.XPM('trainxval_rmsd.xpm')
txv.array.shape

In [ ]:
# minima per row -- for each configuration in train, how far is the nearest one from validation
txv_min = np.min(txv.array,axis=1)
txv_min.shape

In [ ]:
plt.hist(txv_min,bins=50)
plt.show()

In [ ]:
# drop similar structures (to validation trajectory) in train trajectory to avoid dataset being biased
txv_difference = 0.05

train_tr = X_train[np.argwhere(txv_min > txv_difference).flat]
train_tr.xyz.shape

In [ ]:
train_tr.save_xtc('tmp_train.xtc')
gmx.rms(s=conf,f='tmp_train.xtc',f2='test.xtc',n='backbone.ndx',m='trainxtest_rmsd.xpm')

In [ ]:
txt = gf.XPM('trainxtest_rmsd.xpm')
txt.array.shape

In [ ]:
txt_min = np.min(txt.array,axis=1)
txt_min.shape

In [ ]:
plt.hist(txt_min,bins=50)
plt.show()

In [ ]:
# ... one more time with test trajectory & test x validation...
txt_difference = 0.05

x_train = train_tr[np.argwhere(txt_min > txt_difference).flat]
x_train.save_xtc('x_train.xtc')

In [ ]:
# test x validation
gmx.rms(f='test.xtc',f2='validate.xtc',s=conf,n='backbone.ndx',m='testxvalidate_rmsd.xpm')

In [ ]:
txv = gf.XPM('testxvalidate_rmsd.xpm')
txv.array.shape

In [ ]:
txv_min = np.min(txv.array,axis=1)
txv_min.shape

In [ ]:
plt.hist(txv_min,bins=50)
plt.show()

In [ ]:
# ... one more time with test trajectory & test x validation...
txv_difference = 0.05

x_test = X_test[np.argwhere(txv_min > txv_difference).flat]
x_test.save_xtc('x_test.xtc')

In [ ]:

# skip thorough RMS
! ln train.xtc x_train.xtc
! ln test.xtc x_test.xtc


#### Recovery

In [ ]:
# recovery

x_train = md.load('../DE-Shaw/x_train.xtc', top=conf)
x_test = md.load('../DE-Shaw/x_test.xtc', top=conf)
x_all = md.load('../DE-Shaw/trpcage_red.xtc', top=conf)

#### Save computetd geometries

In [ ]:
# get shapes of filtered trajectories that are to be used as datasets
validate_tr = md.load('../DE-Shaw/validate.xtc', top=conf)

trajs = [x_train, validate_tr, x_test, x_all]
x_train.xyz.shape, validate_tr.xyz.shape, x_test.xyz.shape, x_all.xyz.shape

In [ ]:
# reshuffle the geometries to get frame last so that we can use vectorized calculations later on
geoms = [ np.moveaxis(t.xyz,0,-1) for t in trajs]
print ([ g.shape for g in geoms ])

In [ ]:
# save geometries

tf.data.Dataset.from_tensor_slices(geoms[0]).save('datasets/geoms/train')
tf.data.Dataset.from_tensor_slices(geoms[1]).save('datasets/geoms/validate')
tf.data.Dataset.from_tensor_slices(geoms[2]).save('datasets/geoms/test')
tf.data.Dataset.from_tensor_slices(geoms[3]).save('dataset/geoms/X_all')

### Internal coordinates computation

Exercise the ASMSA library on your input. Just check everything appears to work.

There are multiple options that can be combined:
* use traditional internal coordinates (bond distances, angles, and dihedrals) or not
* include additional distances between atoms that may not be bound to express protein folding state more directly
   * dense (all-to-all) atom distances, feasible for very small peptides only
   * sparse atom distances (only some pairs are chosen)
   
* We save the computed internal coordinates for training, and a feature extraction model here, therefore everything in the other notebooks should work too.


#### Compute atom indexes : 
* Backbone
* ON (bacbone Oxigens, Nitrogens)
* Polar atoms
* Alpha Carbons
* Alpha-Beta Carbons

In [ ]:
pdb_path = conf

backbone = []
with open(pdb_path) as f:
    atom_counter = 0
    for line in f:
        if not line.startswith("ATOM"):
            continue
        name = line[12:16].strip()
        if name == "N" or name == "C" or name=="CA":
            backbone.append(atom_counter)
        atom_counter += 1

ON = []
with open(pdb_path) as f:
    atom_counter = 0
    for line in f:
        if not line.startswith("ATOM"):
            continue
        name = line[12:16].strip()
        if name == "N" or name == "O":
            ON.append(atom_counter)
        atom_counter += 1

polar = []
with open(pdb_path) as f:
    atom_counter = 0
    for line in f:
        if not line.startswith("ATOM"):
            continue
        name = line[12:16].strip()
        pol = {
            "N", #backbone amide nitrogen
            "O", #backbone carbonyl oxygen
            "OG", #Serine
            "OG1", #Threonine
            "OH2", #Tyrosine
            "SG", #Cysteine
            "OD1", #Aspartate
            "OD2", #Aspartate
            "OE1", #Glutamate
            "OE2", #Glutamate 
            "ND2", #Asparagine
            "OD1", #Asparagine
            "NE2", #Glutamine
            "OE1", #Glutamine
            "ND1", #Histidine
            "NE2", #Histidine
            "NZ", #Lysine
            "NE", #Arginine
            "NH1", #Arginine
            "NH2", #Arginine
        }
        if name in pol:
            polar.append(atom_counter)
        atom_counter += 1

            
alpha = []
with open(pdb_path) as f:
    atom_counter = 0
    for line in f:
        if not line.startswith("ATOM"):
            continue
        name = line[12:16].strip()
        if name == "CA":
            alpha.append(atom_counter)
        atom_counter += 1

alphabeta = []
with open(pdb_path) as f:
    atom_counter = 0
    for line in f:
        if not line.startswith("ATOM"):
            continue
        name = line[12:16].strip()
        if name == "CA" or name == "CB":
            alphabeta.append(atom_counter)
        atom_counter += 1

print(f'Backbone({len(backbone)}): {backbone}')
print(f'ON({len(ON)}): {ON}')
print(f'Polar Atoms ({len(polar)}): {polar}')
print(f'Alpha C ({len(alpha)}): {alpha}')
print(f'Alpha and Beta ({len(alphabeta)}): {alphabeta}')

#### Extract:
* Bonds
* Angles
* Dihedrals
* Dihedrlas: phi-psi only

In [ ]:
bonds = np.array([[backbone[i], backbone[i+1]] for i in range(len(backbone) - 1)])
angles = np.array([[backbone[i], backbone[i+1], backbone[i+2]] for i in range(len(backbone) - 2)])
dih = np.array([backbone[i:i+4] for i in range(len(backbone) - 3)])

In [ ]:
#!!! TO BE CHECKED 

mask = np.ones(len(dih), dtype=bool)

mask[1] = False

for i in range(2, len(mask)):
    mask[i] = ((i-2) % 3) != 2
phipsi = dih[mask]

#### Compute distances:
* 1) **Choose** the atom selection (e.g atoms = ON): the atoms will be considered for the random pick with the density selection.
  2) **If** "sparse" choose the density.
  3) **If** "dense" all the atoms will be considered.
  

In [ ]:
sparse_dists = asmsa.NBDistancesSparse(geoms[0].shape[0], density=5,  atoms = ON)
dense_dists = asmsa.NBDistancesDense(geoms[0].shape[0])

#### Biuld:

In [ ]:
#mol=asmsa.Molecule(pdb=conf,top=topol,fms=[dense_dists])
mol=asmsa.Molecule(pdb=conf,n_atoms=geoms[0].shape[0],
                   diheds=phipsi,
                   fms=[sparse_dists]) 

In [ ]:
mol_model = mol.get_model()

example_input = torch.randn((*geoms[0].shape[:2],1))

In [ ]:
len(sparse_dists.bonds)
#len(dense_dists.bonds)

In [ ]:
mol.describe_features()

#### Save the features (molecule) model

In [ ]:
mol_model = mol.get_model()

example_input = torch.randn((*geoms[0].shape[:2],1))
traced_script_module = torch.jit.trace(mol_model, example_input)

traced_script_module.save('features.pt')

#### Compute the interanal coordinates now

In [ ]:
intcoords = [ mol.intcoord(g).T for g in geoms]
print(
    [ g.shape for g in geoms ],
    [ i.shape for i in intcoords ]
)

In [ ]:
[train,validate,test, X_all] = intcoords

In [ ]:
geoms[3].shape

In [ ]:
d = mol_model.dihed4_model(torch.from_numpy(geoms[2])).numpy()

In [ ]:
d.shape,test.shape

In [ ]:
def forward(self, geoms):
#        geoms = input.reshape(self.n_atoms, 3, -1)
    a12 = geoms[self.atoms[:, 1]] - geoms[self.atoms[:, 0]]
    a23 = geoms[self.atoms[:, 2]] - geoms[self.atoms[:, 1]]
    a34 = geoms[self.atoms[:, 3]] - geoms[self.atoms[:, 2]]

#        a12 = torch.nn.functional.normalize(a12, p=2, dim=1)
#        a23 = torch.nn.functional.normalize(a23, p=2, dim=1)
#        a34 = torch.nn.functional.normalize(a34, p=2, dim=1)

    vp1 = torch.nn.functional.normalize(torch.cross(a12,a23,axis=1))
    vp2 = torch.nn.functional.normalize(torch.cross(a23,a34,axis=1))
    vp3 = torch.nn.functional.normalize(torch.cross(vp1,a23,axis=1))

    sp1 = torch.sum(vp1 * vp2, axis=1)
    sp2 = torch.sum(vp3 * vp2, axis=1)

    """ original:
    # output for i-th dihedral angle
        aa = np.arctan2(sp1,sp2) - np.pi * .5
        return np.sin(aa), np.cos(aa)
    """

    #NOTE: Why adding two variables that determine each other? It the angle better?
    # return torch.nn.functional.normalize(torch.stack([-sp2, sp1]), p=2, dim=0).reshape(2*len(self.atoms), geoms.shape[2])
    return sp1,sp2
    #return torch.stack([-sp2, sp1]).reshape(2*len(self.atoms), geoms.shape[2])


In [ ]:
sp1,sp2 = forward(mol_model.dihed4_model,torch.from_numpy(geoms[2]))

In [ ]:
sp1.shape,sp2.shape

In [ ]:
torch.stack([-sp2, sp1]).reshape((146,-1)).shape

In [ ]:
a1 = torch.from_numpy(np.array([[1,2,3],[4,5,6]]))
a1

In [ ]:
a1.shape

In [ ]:
a2 = a1+.2
a2

In [ ]:
torch.stack([a1,a2]).reshape(4,-1)

In [ ]:
# validate the saved model -- should yield nearly 0.

test_from_model = mol_model(torch.from_numpy(geoms[2])).numpy()
np.max(test - test_from_model.T)

In [ ]:
test_from_model.shape

In [ ]:
# normalize training set
train_mean = np.mean(train,axis=0)
train -= train_mean
train_scale = np.std(train,axis=0)
train /= train_scale

In [ ]:
# normalize test and validation sets
test -= train_mean
test /= train_scale
validate -= train_mean
validate /= train_scale
X_all -= train_mean
X_all /= train_scale

#### Save the features (molecule) model

In [ ]:
mol_model = mol.get_model()

example_input = torch.randn((*geoms[0].shape[:2],1))
traced_script_module = torch.jit.trace(mol_model, example_input)

traced_script_module.save('features.pt')

#### Save internal coordinates as datasets which can be loaded in **train.ipynb** and **tune.ipynb** notebooks 

In [ ]:
# save for usage in tune/train/test phase

tf.data.Dataset.from_tensor_slices(train).save('datasets/intcoords/train')
tf.data.Dataset.from_tensor_slices(validate).save('datasets/intcoords/validate')
tf.data.Dataset.from_tensor_slices(test).save('datasets/intcoords/test')
tf.data.Dataset.from_tensor_slices(X_all).save('datasets/intcoords/X_all')

np.savetxt('datasets/intcoords/mean.txt',train_mean)
np.savetxt('datasets/intcoords/scale.txt',train_scale)

# Density of the conformational space

- Sample the training trajectory randomly
- For each point in the trajectory:
  - calculate RMSD to all points in the sample
  - pick some number $n$ of nearest ones
  - calculate the _density_ at this point as $$ d = \sum_{i=1}^n e^{-d_i} / n $$  i.e. the nearer the sample points are, the higher the density
 
Altogether, $d$ roughly corresponds to the probability that the molecule during simulation ends up in this area of the conformational space.

In [ ]:
sample_size = 5000
x_train = md.load('x_train.xtc', top=conf)
tr_sample = x_train[np.random.choice(len(x_train),sample_size,False)]
tr_sample.save('sample.xtc')

In [ ]:
gmx.rms(f='x_train.xtc',f2='sample.xtc',s=conf,n='backbone.ndx',m='sample_rmsd.xpm')

In [ ]:
rms = gf.XPM('sample_rmsd.xpm')

#### Visual check to verify the sample size is representative
- typically, not many distances should be less than 0.1 nm and more than 1 nm 
(the latter depends on the molecule, can be more for e.g. big disordered proteins)
- the histogram should be semi-smooth

In [ ]:
plt.hist(rms.array.flatten(),bins=50)
plt.show()

In [ ]:
k_nearest = 200
rms_sort = np.sort(rms.array.astype(np.float32))
erms = np.exp(-rms_sort[:,:k_nearest])
dens = (np.sum(erms,axis=1)-1.) / (erms.shape[1] - 1)

#### Histogram of densities
- quite high number of points should fall above 0.8, those are low energy basins
- the interval [0.5, 1.0] should be reasonably covered
- on the contrary, too many points below 0.4 would indicate either insufficient sampling above or too sparse trajectory

In [ ]:
plt.hist(dens,bins=20)
plt.show()

In [ ]:
len(dens),len(x_train)

In [ ]:
np.savetxt('datasets/train_density.txt',dens)